In [1]:
import os, sys
home_path = os.path.expanduser("~")
sys.path.append(os.path.join(home_path, "github", "Thunder", "Binance"))

In [59]:
import SystemConfig

symbols = tuple(SystemConfig.Streaming.symbols)

from typing import Any, List
import json


class CalculationReport:
    """
    분석용 계산결과를 저장하는 클라스이다.
    지정된 속성값은 없으며, 업데이트대마다 전부 신규로 등록처리한다.
    """
    
    def __init__(self):
        ...
        
    def set_data(self, attr_name:str, data:Any):
        setattr(self, attr_name, data)
    
    def get_data(self, attr_name:str) -> Any:
        if not isinstance(attr_name, str):
            raise ValueError(f"타입 입력 오류: {type(attr_name)}")
        if not hasattr(self, attr_name):
            raise ValueError(f"name 입력 오류: {attr_name}")
            
        return getattr(self, attr_name)
    
    def get_fields(self) -> List:
        return [key for key in vars(self).keys()]
    
    def to_dict(self):
        result = {}
        for attr in self.__dict__:
            result[attr] = getattr(self, attr)
        return result
    
    def clear(self):
        for attr in self.__dict__:
            delattr(self, attr)
    
    def __str__(self) -> str:
        return json.dumps(vars(self), indent=2)

    def __repr__(self):
        return str(self.to_dict())


class ComputedResults:
    __slots__ = symbols
    def __init__(self):
        for slot in self.__slots__:
            setattr(self, slot, CalculationReport())
    
    def to_dict(self):
        result = {}
        for slot in self.__slots__:
            result[slot] = slot.to_dict()
        return result

    def __repr__(self):
        return str(self.to_dict())

In [60]:
dummy = CalculationReport()

In [61]:
dummy.set_data("BTCUSDT",[123,123,123])

In [64]:
str(dummy)

'{\n  "BTCUSDT": [\n    123,\n    123,\n    123\n  ]\n}'